Data Cleaning Phase

In [2]:
import pandas as pd

In [ ]:
dataset = pd.read_csv('vscode_bugs.csv')
print(dataset.head())
print(f"The data types in the rows are: \n {dataset.info()}")
print(f"Summary statistics: {dataset.describe()}")

   Issue id                                            Summary  \
0    223706  Configure unassigned keybindings, command name...   
1    223658  Some suggestions are missing on Windows with pwsh   
2    223641  Cannot split a terminal without a group [objec...   
3    223622  SCM - history graph handling of first commit i...   
4    223607          SCM - history graph using incorrect color   

                     Created                   Resolved  \
0  2024-07-25 14:34:59+00:00                        NaN   
1  2024-07-25 13:06:40+00:00  2024-07-25 16:14:57+00:00   
2  2024-07-25 12:19:09+00:00                        NaN   
3  2024-07-25 10:44:47+00:00                        NaN   
4  2024-07-25 09:43:02+00:00  2024-07-25 11:02:46+00:00   

                                         Description Resolution  
0  - Place cursor in the panel chat\r\n- Open the...        NaN  
1  I was trying to verify https://github.com/micr...  completed  
2  ![image](https://github.com/user-attachments/a.